<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Temporal-RangeList-Queries" data-toc-modified-id="Temporal-RangeList-Queries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Temporal RangeList Queries</a></span><ul class="toc-item"><li><span><a href="#1-on-1-Panel-Interviews" data-toc-modified-id="1-on-1-Panel-Interviews-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1-on-1 Panel Interviews</a></span><ul class="toc-item"><li><span><a href="#Chris-Hayes-and-Bernie-Sanders-in-the-same-frame" data-toc-modified-id="Chris-Hayes-and-Bernie-Sanders-in-the-same-frame-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Chris Hayes and Bernie Sanders in the same frame</a></span></li><li><span><a href="#Full-interviews" data-toc-modified-id="Full-interviews-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Full interviews</a></span><ul class="toc-item"><li><span><a href="#Only-look-at-videos-where-Chris-Hayes-and-Bernie-Sanders-appear-together" data-toc-modified-id="Only-look-at-videos-where-Chris-Hayes-and-Bernie-Sanders-appear-together-1.1.2.1"><span class="toc-item-num">1.1.2.1&nbsp;&nbsp;</span>Only look at videos where Chris Hayes and Bernie Sanders appear together</a></span></li><li><span><a href="#TemporalRangeList-API" data-toc-modified-id="TemporalRangeList-API-1.1.2.2"><span class="toc-item-num">1.1.2.2&nbsp;&nbsp;</span>TemporalRangeList API</a></span></li><li><span><a href="#TemporalRangeList-Round-2" data-toc-modified-id="TemporalRangeList-Round-2-1.1.2.3"><span class="toc-item-num">1.1.2.3&nbsp;&nbsp;</span>TemporalRangeList Round 2</a></span></li></ul></li></ul></li><li><span><a href="#5-short-consecutive-shots" data-toc-modified-id="5-short-consecutive-shots-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>5 short consecutive shots</a></span></li></ul></li></ul></div>

# Temporal RangeList Queries

This notebook explores using the new TemporalRangeList API to compose more complicated temporal queries.

## 1-on-1 Panel Interviews

First, let's look for panel interviews. These are interviews where the host and interview subject are in different places, so their faces are side by side in a panel arrangement. Here's a good example of an interview between Chris Hayes and Beto O'Rourke: https://youtu.be/j0hwHmofc6w?t=66.

### Chris Hayes and Bernie Sanders in the same frame

Let's start by looking for an interview between Chris Hayes and Bernie Sanders. First, we get all the frames where both of them appear together.

In [ ]:
from esper.stdlib import *
from esper.prelude import *
from esper.spark_util import *
from pyspark.sql.functions import *
from esper.temporal_predicates import *
from esper.temporal_rangelist import TemporalRange, TemporalRangeList

import numpy as np
import sys

In [ ]:
face_identities = get_face_identities(include_bbox=True, include_name=True)
print('Schema: ', face_identities)

In [ ]:
chris_hayes = face_identities.filter(
    face_identities.name == 'chris hayes').filter(
    'probability > 0.99').alias('chris_hayes')
bernie_sanders = face_identities.filter(
    face_identities.name == 'bernie sanders').filter(
    'probability > 0.99').alias('bernie_sanders')

chris_hayes_and_bernie_sanders = chris_hayes.join(
    bernie_sanders,
    (chris_hayes.video_id == bernie_sanders.video_id) &
    (chris_hayes.min_frame == bernie_sanders.min_frame) &
    (chris_hayes.max_frame == bernie_sanders.max_frame)
).select(
    col('chris_hayes.id').alias('a_id'),
    col('bernie_sanders.id').alias('b_id'),
    col('chris_hayes.video_id'),
    col('chris_hayes.min_frame'),
    col('chris_hayes.max_frame'),
    col('chris_hayes.fps'),
    col('chris_hayes.bbox_x1').alias('a_bbox_x1'),
    col('chris_hayes.bbox_x2').alias('a_bbox_x2'),
    col('chris_hayes.bbox_y1').alias('a_bbox_y1'),
    col('chris_hayes.bbox_y2').alias('a_bbox_y2'),
    col('bernie_sanders.bbox_x1').alias('b_bbox_x1'),
    col('bernie_sanders.bbox_x2').alias('b_bbox_x2'),
    col('bernie_sanders.bbox_y1').alias('b_bbox_y1'),
    col('bernie_sanders.bbox_y2').alias('b_bbox_y2'))

print(chris_hayes_and_bernie_sanders)
print('Count: ', chris_hayes_and_bernie_sanders.count())

In [ ]:
# Now let's display these results in an Esper widget

both_faces = chris_hayes_and_bernie_sanders.collect()

materialized_results = []
for shot in both_faces[:100]:
    materialized_results.append({
        'video': shot.video_id,
        'min_frame': shot.min_frame,
        'objects': [{
            'id': shot.a_id,
            'type': 'bbox',
            'bbox_x1': shot.a_bbox_x1,
            'bbox_x2': shot.a_bbox_x2,
            'bbox_y1': shot.a_bbox_y1,
            'bbox_y2': shot.a_bbox_y2
        }, {
            'id': shot.b_id,
            'type': 'bbox',
            'bbox_x1': shot.b_bbox_x1,
            'bbox_x2': shot.b_bbox_x2,
            'bbox_y1': shot.b_bbox_y1,
            'bbox_y2': shot.b_bbox_y2
        }]
    })
esper_widget(simple_result(materialized_results, 'Faces'))

### Full interviews

Now that we have all the frames where they appear together, it would be great if we could get the full interviews.

First, we're going to filter the Bernie Sanders and Chris Hayes tables so that we only have rows that correspond to the videos where they both appear. Then we're going to use our new TemporalRangeList API to coalesce appearances of Bernie Sanders + Chris Hayes with the Bernie Sanders appearances to get the full interviews.

#### Only look at videos where Chris Hayes and Bernie Sanders appear together
First, we filter out all the videos where Bernie Sanders and Chris Hayes do not appear together.

In [ ]:
bernie_sanders_relevant = bernie_sanders.join(
    chris_hayes_and_bernie_sanders,
    bernie_sanders.video_id == chris_hayes_and_bernie_sanders.video_id
).select('bernie_sanders.*').dropDuplicates().alias('bernie_sanders_relevant')

print(bernie_sanders_relevant)
print(bernie_sanders_relevant.count())

In [ ]:
bernie_sanders_materialized = bernie_sanders_relevant.collect()

In [ ]:
# Group by video ID

both_faces_dict = {}
bernie_dict = {}

for row in both_faces:
    if row.video_id in both_faces_dict:
        both_faces_dict[row.video_id].append(row)
    else:
        both_faces_dict[row.video_id] = [row]
for row in bernie_sanders_materialized:
    if row.video_id in bernie_dict:
        bernie_dict[row.video_id].append(row)
    else:
        bernie_dict[row.video_id] = [row]

#### TemporalRangeList API

Now, we use our TemporalRangeList API to coalesce all appearances of Bernie Sanders with Chris + Bernie.

In [ ]:
# Now we use our TemporalRangeList API to coalesce Bernie + Chris Hayes with Bernie appearances

video_ids = both_faces_dict.keys()
full_interviews = {}

for video in video_ids:
    fps = bernie_dict[video][0].fps
    bernie_list = TemporalRangeList([(row.min_frame, row.max_frame, 1)
                                     for row in bernie_dict[video]])
    bernie_and_chris_list = TemporalRangeList([(row.min_frame, row.max_frame, 2) 
                                               for row in both_faces_dict[video]])
    everything_list = bernie_and_chris_list.dilate(5).merge(bernie_list, predicate=Overlaps())
    three_seconds = 3 * fps
    everything_list = everything_list.coalesce().filter_length(min_length=three_seconds)
    
    full_interviews[video] = [{'min_frame': tr.get_start(), 'max_frame': tr.get_end()}
                              for tr in everything_list.get_temporal_ranges()]

In [ ]:
# Finally, display the results in the Esper widget!
materialized_results = {}
full_count = 0
for video in full_interviews:
    if len(full_interviews[video]) == 0:
        continue
    materialized_results[video] = [
        {'track': video, 'min_frame': row['min_frame'],
         'max_frame': row['max_frame'], 'video': video}
        for row in full_interviews[video]]
    full_count += len(materialized_results[video])

groups = [{
    'type': 'contiguous',
    'label': video,
    'elements': materialized_results[video]
} for video in sorted(materialized_results.keys())]
        
esper_widget({'result': groups, 'count': full_count, 'type': 'Interviews'})

#### TemporalRangeList Round 2

Our first attempt could use some refining... we ran into a lot of cases where we just had the cases where Bernie and Chris Hayes were both on screen. Let's try to filter so that we only look at cases where there's at least one cut from both of them to just Bernie

In [ ]:
video_ids = both_faces_dict.keys()
interviews_with_cuts = {}

for video in video_ids:
    fps = bernie_dict[video][0].fps
    bernie_list = TemporalRangeList([(row.min_frame, row.max_frame, 1)
                                     for row in bernie_dict[video]])
    bernie_and_chris_list = TemporalRangeList([(row.min_frame, row.max_frame, 2) 
                                               for row in both_faces_dict[video]])
    
    bernie_alone = bernie_list.minus(bernie_and_chris_list).filter_length(min_length=fps)
    bernie_next_to_both = bernie_alone.merge(bernie_and_chris_list,
                                             predicate=Or(Before(0, fps), After(0, fps)))
    
    everything_list = bernie_next_to_both.dilate(
        5).coalesce().filter_length(min_length = 60 * fps)
    if len(everything_list.get_temporal_ranges()) > 0:
        interviews_with_cuts[video] = [{'min_frame': tr.get_start(), 'max_frame': tr.get_end()}
                              for tr in everything_list.get_temporal_ranges()]

In [ ]:
# Display the results in the Esper widget!
def results_for_esper(clips_by_video):
    materialized_results = {}
    full_count = 0
    for video in clips_by_video:
        if len(clips_by_video[video]) == 0:
            continue
        materialized_results[video] = [
            {'track': video, 'min_frame': row['min_frame'],
             'max_frame': row['max_frame'], 'video': video}
            for row in clips_by_video[video]]
        full_count += len(materialized_results[video])

    groups = [{
        'type': 'contiguous',
        'label': video,
        'elements': materialized_results[video]
    } for video in sorted(materialized_results.keys())]

    return {'result': groups, 'count': full_count, 'type': 'Interviews'}

In [ ]:
esper_widget(results_for_esper(interviews_with_cuts))

Those look better. Let's dig in a little deeper to make sure we get the full clips.

In [ ]:
average_length = np.mean([clip['max_frame'] - clip['min_frame'] for v in interviews_with_cuts.values() for clip in v])
print('Average number of frames: ', average_length)

In [ ]:
print(interviews_with_cuts)

## 5 short consecutive shots

Give me all situations where there were five short consecutive shots

In [ ]:
shots = get_shots()
short_shots = shots.filter('duration < 0.5').orderBy('video_id')
print(short_shots.count())

In [ ]:
short_shots_by_vid = short_shots.select('video_id', 'min_frame', 'max_frame').groupBy('video_id').agg(
    collect_list('min_frame').alias('min_frames'), collect_list('max_frame').alias('max_frames'))
short_shots_by_vid.show()
print(short_shots_by_vid.count())

In [ ]:
collected_shots = {}
for idx, row in enumerate(short_shots_by_vid.collect()):
    if (idx % 1000) == 0:
        if (idx % 10000) == 0:
            print()
            print(idx)
        sys.stdout.write('.')
    video = row.video_id
    shots_in_video = zip(row.min_frames, row.max_frames)
    trlist = TemporalRangeList([(shot[0], shot[1], 1) for shot in shots_in_video]).dilate(1)
    five_consecutive_shots = trlist.merge(
        trlist.merge(
            trlist.merge(
                trlist.merge(trlist, predicate=MeetsBefore()),
                predicate=MeetsBefore()),
            predicate=MeetsBefore()),
        predicate=MeetsBefore()).get_temporal_ranges()
    if len(five_consecutive_shots) > 0:
        collected_shots[video] = five_consecutive_shots

print(collected_shots)

It turns out this doesn't happen...